# Set Up
If you haven't setup your credentials in the config.json file, please do so in the given prompt.

In [1]:
import pandas as pd
import requests
import json

In [23]:
# Setting up the config for usage.
try:
    with open('config.json', 'r') as f:
        config = json.load(f)
    user_agent = config['user_agent']
    key1 = config['key1']
    key2 = config['key2']
    key3 = config['key3']
    username = config['username']
    print("Config Loaded Successfully.")
    key_list = [key1, key2, key3]
    
except:
    print('config.json wasn\'t found.\nCreating new config file. Please enter your credentials.')
    user_agent = input('Enter User Agent: ')
    username = input('Enter your username: ')
    key1 = input('Enter Secret Key: ')
    key2 = input('Enter Secret Key 2:\nLeave blank if you don\'t have mirror keys. ')
    if key2 == '':
        key2 = key1
    key3 = input('Enter Secret Key 3:\nLeave blank if you don\'t have mirror keys. ')
    if key3 == '':
        key3 = key1
    
#     with open('config.json.sample','r') as f:
#         config = json.load(f)
    config = {'user_agent':None,'key1':None, 'key2':None, 'key3':None, 'username':None}
    config['user_agent'] = user_agent
    config['key1'] = key1
    config['key2'] = key2
    config['key3'] = key3
    config['username'] = username
    
    with open('config.json','w+') as f:
        json.dump(config, f)
    
    key_list = [key1, key2, key3]

    print()
    print("Your configuration has been saved to config.json!")
    
# Fetches variables: user_agent, username, key1, key2, key3, key_list

Config Loaded Successfully.


# Fetching Data from Last.fm

In [3]:
# Generic Function for sending requests

def send_request(user=username, page=1, limit=200, key=key1, method='user.getrecenttracks'):
    
    headers = {'user-agent': user_agent}
    
    payload ={
    'limit':limit,
    'user':user,
    'page': page,
    'api_key': key,
    'method': method,
    'format': 'json'
    }
    
    api_reply = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=payload)
    
    return api_reply

In [4]:
# Fetch a page and clean data before return.
# Also handles exceptions.

def get_page(num=1, username=username, key=key1, method='user.getrecenttracks'):
    
    try:
        response = send_request(page=num, key=key, method=method)
        
        if response.status_code != 200:
            raise Exception(api_reply.text)
        
        page = (response.json())['recenttracks']['track']
        
        if num == 1: 
            global totalPages
            totalPages = int((response.json())['recenttracks']['@attr']['totalPages'])
        
        if (num!=1) & ('@attr' in page[0].keys()): 
            if (page[0]['@attr']=={'nowplaying': 'true'}):
                del page[0]
        
        return page

    except:
        raise Exception("request status {}".format(response.status_code))

# Get the total number of pages for 200 scrobbles per page.

def get_totalPages(method='user.getrecenttracks'):
    response = send_request(method = method)
    totalPages = int((response.json())['recenttracks']['@attr']['totalPages'])
    return totalPages

totalPages = get_totalPages()

In [5]:
# Picks a key from given list of keys
def key_picker(num, ls):
    index_num = num % len(ls)
    return ls[index_num]

In [6]:
# Fetches all the pages and merges them into a list "pages"
# Uses a fancy progress bar to show progress.
# Large accounts may time time to fetch!


from IPython.core.display import clear_output

pages = []

for n in range(1, (totalPages)+1):
    
    print('fetched page {}/{}'.format(n, totalPages))
    
    spaces = (((totalPages-n)/totalPages)*50)
    bar = ((n/totalPages)*50)
    string = '[{}{}] {}% Completed'.format('~'*int(bar),' '*int(spaces), int(bar)*2)
    print(string)
    
    clear_output(wait=True)
    
    ky = key_picker(num = n, ls = key_list)
    fetched_page = get_page(num=n, key=ky)
    
    pages = [*pages, *fetched_page]
print()
print("Fetched {} pages, {} scrobbles".format(totalPages, len(pages)))


Fetched 63 pages, 12445 scrobbles


# Cleaning up Fetched Data with #Pandas

In [7]:
df = pd.DataFrame(pages)
df.head()

,artist,streamable,image,mbid,album,name,@attr,url,date
0,{'mbid': '52db98c1-0cc9-4aab-9f2d-ea8d359234f3...,0,"[{'size': 'small', '#text': 'https://lastfm.fr...",,{'mbid': 'bdb7c15c-cf46-4c6e-9ba3-71dfb1c820e3...,Subha Hone Na De,{'nowplaying': 'true'},https://www.last.fm/music/Pritam/_/Subha+Hone+...,NaN
1,"{'mbid': '', '#text': 'A.R. Rahman'}",0,"[{'size': 'small', '#text': 'https://lastfm.fr...",,"{'mbid': '', '#text': 'Rang De Basanti (Origin...",Luka Chuppi,NaN,https://www.last.fm/music/A.R.+Rahman/_/Luka+C...,"{'uts': '1643062361', '#text': '24 Jan 2022, 2..."
2,{'mbid': '485f576c-6ade-4c0f-9406-34720f15fb02...,0,"[{'size': 'small', '#text': 'https://lastfm.fr...",,"{'mbid': '', '#text': 'Nanchaku'}",Nanchaku,NaN,https://www.last.fm/music/Seedhe+Maut/_/Nanchaku,"{'uts': '1643062167', '#text': '24 Jan 2022, 2..."
3,{'mbid': '6b0b4484-442e-489d-bb8e-5e79dcd8e5f4...,0,"[{'size': 'small', '#text': 'https://lastfm.fr...",,"{'mbid': '', '#text': 'Race 2 (Original Motion...",Lat Lag Gayee,NaN,https://www.last.fm/music/Benny+Dayal/_/Lat+La...,"{'uts': '1643061887', '#text': '24 Jan 2022, 2..."
4,{'mbid': 'ed3f4831-e3e0-4dc0-9381-f5649e9df221...,0,"[{'size': 'small', '#text': 'https://lastfm.fr...",,"{'mbid': '', '#text': 'Raees'}",Zaalima,NaN,https://www.last.fm/music/Arijit+Singh/_/Zaalima,"{'uts': '1643061588', '#text': '24 Jan 2022, 2..."


In [8]:
df.drop(['image','@attr','streamable'], inplace=True, axis=1)
df.head()

,artist,mbid,album,name,url,date
0,{'mbid': '52db98c1-0cc9-4aab-9f2d-ea8d359234f3...,,{'mbid': 'bdb7c15c-cf46-4c6e-9ba3-71dfb1c820e3...,Subha Hone Na De,https://www.last.fm/music/Pritam/_/Subha+Hone+...,NaN
1,"{'mbid': '', '#text': 'A.R. Rahman'}",,"{'mbid': '', '#text': 'Rang De Basanti (Origin...",Luka Chuppi,https://www.last.fm/music/A.R.+Rahman/_/Luka+C...,"{'uts': '1643062361', '#text': '24 Jan 2022, 2..."
2,{'mbid': '485f576c-6ade-4c0f-9406-34720f15fb02...,,"{'mbid': '', '#text': 'Nanchaku'}",Nanchaku,https://www.last.fm/music/Seedhe+Maut/_/Nanchaku,"{'uts': '1643062167', '#text': '24 Jan 2022, 2..."
3,{'mbid': '6b0b4484-442e-489d-bb8e-5e79dcd8e5f4...,,"{'mbid': '', '#text': 'Race 2 (Original Motion...",Lat Lag Gayee,https://www.last.fm/music/Benny+Dayal/_/Lat+La...,"{'uts': '1643061887', '#text': '24 Jan 2022, 2..."
4,{'mbid': 'ed3f4831-e3e0-4dc0-9381-f5649e9df221...,,"{'mbid': '', '#text': 'Raees'}",Zaalima,https://www.last.fm/music/Arijit+Singh/_/Zaalima,"{'uts': '1643061588', '#text': '24 Jan 2022, 2..."


In [9]:
#Split the artist column, and rename the df. Drop the orginal column
artist_split = df['artist'].apply(pd.Series).rename(columns = {'mbid':'artist_mbid','#text':'artist'})
df.drop(columns = ['artist'], inplace=True)

#Split the album column, and rename the df. Drop the original column
album_split = df['album'].apply(pd.Series).rename(columns = {'mbid':'album_mbid','#text':'album'})
df.drop(columns = ['album'], inplace=True)

#Split the date column, and rename the df. Drop the original column
#Also check if residual '0' column is made. (If the top song is being streamed, it wont include the timing.)
date_split = df['date'].apply(pd.Series).rename(columns = {'uts':'uts','#text':'date'})
df.drop(columns = ['date'], inplace=True)
if 0 in date_split.columns:
    date_split.drop(0, axis=1, inplace=True)

#Merging the new columns into main df
df = pd.concat([df, artist_split, album_split, date_split],axis=1)
df.head()

,mbid,name,url,artist_mbid,artist,album_mbid,album,date,uts
0,,Subha Hone Na De,https://www.last.fm/music/Pritam/_/Subha+Hone+...,52db98c1-0cc9-4aab-9f2d-ea8d359234f3,Pritam,bdb7c15c-cf46-4c6e-9ba3-71dfb1c820e3,Desi Boyz,NaN,NaN
1,,Luka Chuppi,https://www.last.fm/music/A.R.+Rahman/_/Luka+C...,,A.R. Rahman,,Rang De Basanti (Original Motion Picture Sound...,"24 Jan 2022, 22:12",1643062361
2,,Nanchaku,https://www.last.fm/music/Seedhe+Maut/_/Nanchaku,485f576c-6ade-4c0f-9406-34720f15fb02,Seedhe Maut,,Nanchaku,"24 Jan 2022, 22:09",1643062167
3,,Lat Lag Gayee,https://www.last.fm/music/Benny+Dayal/_/Lat+La...,6b0b4484-442e-489d-bb8e-5e79dcd8e5f4,Benny Dayal,,Race 2 (Original Motion Picture Soundtrack),"24 Jan 2022, 22:04",1643061887
4,,Zaalima,https://www.last.fm/music/Arijit+Singh/_/Zaalima,ed3f4831-e3e0-4dc0-9381-f5649e9df221,Arijit Singh,,Raees,"24 Jan 2022, 21:59",1643061588


In [10]:
#Reordering columns
col_reorder = ['artist','name','album','date','artist_mbid','mbid','album_mbid','uts','url']
df = df.reindex(col_reorder, axis=1)
df.head()

,artist,name,album,date,artist_mbid,mbid,album_mbid,uts,url
0,Pritam,Subha Hone Na De,Desi Boyz,NaN,52db98c1-0cc9-4aab-9f2d-ea8d359234f3,,bdb7c15c-cf46-4c6e-9ba3-71dfb1c820e3,NaN,https://www.last.fm/music/Pritam/_/Subha+Hone+...
1,A.R. Rahman,Luka Chuppi,Rang De Basanti (Original Motion Picture Sound...,"24 Jan 2022, 22:12",,,,1643062361,https://www.last.fm/music/A.R.+Rahman/_/Luka+C...
2,Seedhe Maut,Nanchaku,Nanchaku,"24 Jan 2022, 22:09",485f576c-6ade-4c0f-9406-34720f15fb02,,,1643062167,https://www.last.fm/music/Seedhe+Maut/_/Nanchaku
3,Benny Dayal,Lat Lag Gayee,Race 2 (Original Motion Picture Soundtrack),"24 Jan 2022, 22:04",6b0b4484-442e-489d-bb8e-5e79dcd8e5f4,,,1643061887,https://www.last.fm/music/Benny+Dayal/_/Lat+La...
4,Arijit Singh,Zaalima,Raees,"24 Jan 2022, 21:59",ed3f4831-e3e0-4dc0-9381-f5649e9df221,,,1643061588,https://www.last.fm/music/Arijit+Singh/_/Zaalima


# Exporting into CSV

In [27]:
df.to_csv('export.csv', encoding='utf-8')

# Data Analysis & Visualization

In [ ]:
# In Progress